In [3]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class BlinkSequenceDataset(Dataset):
    def __init__(self, csv_file, seq_len=20):
        df = pd.read_csv(csv_file).dropna()
        self.seq_len = seq_len
        features = ['ratio', 'ratio_avg', 'distance_vertical', 'distance_horizontal']
        self.X = df[features].values.astype('float32')
        self.y = df['manual_blink'].values.astype('int64')
    def __len__(self):
        return len(self.y) - self.seq_len + 1
    def __getitem__(self, idx):
        x_seq = self.X[idx:idx+self.seq_len]
        y_seq = self.y[idx+self.seq_len-1]  # Predict for the last row in the sequence
        return torch.tensor(x_seq), torch.tensor(y_seq)

In [14]:
import torch.nn as nn

class BlinkLSTMNet(nn.Module):
    def __init__(self, input_size=4, hidden_size=96, num_layers=5, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # x: (batch, seq_len, input_size)
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Take output from last time step
        out = self.fc(out)
        return out

In [15]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

dataset = BlinkSequenceDataset('blink_data_20250528_174203.csv', seq_len=20)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = BlinkLSTMNet()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(30):
    total_loss = 0
    for X, y in dataloader:
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

torch.save(model.state_dict(), "blink_lstm_model.pth")

Epoch 1, Loss: 0.5207
Epoch 2, Loss: 0.3939
Epoch 3, Loss: 0.3822
Epoch 4, Loss: 0.3714
Epoch 5, Loss: 0.3675
Epoch 6, Loss: 0.3601
Epoch 7, Loss: 0.3360
Epoch 8, Loss: 0.3286
Epoch 9, Loss: 0.3535
Epoch 10, Loss: 0.3295
Epoch 11, Loss: 0.3334
Epoch 12, Loss: 0.3400
Epoch 13, Loss: 0.3050
Epoch 14, Loss: 0.3013
Epoch 15, Loss: 0.3333
Epoch 16, Loss: 0.2919
Epoch 17, Loss: 0.2831
Epoch 18, Loss: 0.3300
Epoch 19, Loss: 0.3448
Epoch 20, Loss: 0.2947
Epoch 21, Loss: 0.2769
Epoch 22, Loss: 0.2934
Epoch 23, Loss: 0.2686
Epoch 24, Loss: 0.2750
Epoch 25, Loss: 0.2769
Epoch 26, Loss: 0.2688
Epoch 27, Loss: 0.2708
Epoch 28, Loss: 0.2607
Epoch 29, Loss: 0.2553
Epoch 30, Loss: 0.2894


In [5]:
import torch
import numpy as np

model = BlinkLSTMNet()
model.load_state_dict(torch.load("blink_lstm_model.pth"))
model.eval()

# x_seq: shape (1, 20, 4)
x_seq = np.random.rand(1, 20, 4).astype('float32')  # Replace with your actual last 20 rows
x_seq = torch.tensor(x_seq)
with torch.no_grad():
    logits = model(x_seq)
    pred = torch.argmax(logits, dim=1)
    print("Blink" if pred.item() == 1 else "Not Blink")

Not Blink


In [5]:
import torch.nn as nn

class BlinkLSTMNet(nn.Module):
    def __init__(self, input_size=4, hidden_size=32, num_layers=1, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out